In [1]:
import pandas as pd
import numpy as np

In [63]:
import re

In [66]:
import requests

In [81]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle
from sklearn.metrics.pairwise import cosine_similarity

In [48]:
df = pd.read_csv('fake.csv')
df = df[df.language=='english']
df.loc[df['text'].isnull(), 'text'] = ''
df.loc[df['thread_title'].isnull(),'thread_title'] = ''

In [37]:
ranks = [i for i in df['domain_rank'].tolist() if np.isnan(i) == False]

In [27]:
participants_frequency = pd.crosstab(index = df['participants_count'], columns='count')

In [28]:
set([i for i in df['type'].tolist()])

{'bias', 'bs', 'conspiracy', 'fake', 'hate', 'junksci', 'satire', 'state'}

In [45]:
df.describe()

,ord_in_thread,domain_rank,spam_score,replies_count,participants_count,likes,comments,shares
count,12403.000000,8228.000000,12403.000000,12403.000000,12403.000000,12403.000000,12403.000000,12403.000000
mean,0.934371,38715.689961,0.024484,1.449649,1.779973,10.416593,0.028300,10.416593
std,6.637846,26978.438752,0.120583,9.881279,7.042666,78.689100,0.543768,78.689100
min,0.000000,486.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,17592.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
50%,0.000000,34478.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
75%,0.000000,61423.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
max,100.000000,98679.000000,1.000000,309.000000,240.000000,988.000000,30.000000,988.000000


## Part One

In [49]:
for column in list(df.columns):
    if df[column].count()<len(df):
        print(column, df[column].count())

author 10194
title 11723
country 12227
domain_rank 8228
main_img_url 8779


In [67]:
r = requests.get('https://raw.githubusercontent.com/stanfordnlp/CoreNLP/master/data/edu/stanford/nlp/patterns/surface/stopwords.txt')
stopwords = set(r.text.split())
more_words = ["t","m", "ll","re","don","doesn"]
stopwords |= set(more_words)

In [68]:
df['text_lower'] = df['text'].map(lambda x: re.sub(r'[^a-z ]','',x.lower()))
df['text_lower'] = df['text_lower'].map(lambda x: ' '.join([word for word in x.strip().split() if word not in stopwords]))

In [76]:
df['thread_lower'][0]

'muslims busted stole millions govt benefits'

In [74]:
df['thread_lower'] = df['thread_title'].map(lambda x: re.sub(r'[^a-z ]','',x.lower()))
df['thread_lower'] = df['thread_lower'].map(lambda x: ' '.join([word for word in x.strip().split() if word not in stopwords]))

In [82]:
### Similarities of two texts
def similarity(t1,t2):
    t = [t1,t2]
    tfidf_vectorizer = TfidfVectorizer()
    vectors = tfidf_vectorizer.fit_transform(t)
    vectors = vectors.toarray()
    sim = cosine_similarity(vectors[0].reshape(1,-1),vectors[1].reshape(1,-1))[0]
    
    return sim

In [86]:
similarity('You will have to arrange to print late poster submissions on your own', 'I highly encourage you to meet with me and discuss your ideas before submitting your draft')

array([ 0.19472336])

GBC with second dataset

In [30]:
df2 = shuffle(pd.read_csv('fake_or_real_news.csv'))
df2_train, df2_test = train_test_split(df2, test_size=0.2, random_state=1)

In [31]:
count_vect = CountVectorizer()
tfidf_transformer = TfidfTransformer()

In [32]:
title_count = count_vect.fit_transform(df2['title'])
title_tfidf = tfidf_transformer.fit_transform(title_count)
(title_tfidf_train, title_tfidf_test, Y_train, Y_test) = train_test_split(title_tfidf, 
                                                                  df2['label'], 
                                                                  test_size=0.2, random_state=1)

In [34]:
gbc = GradientBoostingClassifier().fit(title_tfidf_train, Y_train)
title_pred = gbc.predict(title_tfidf_test)
confusion_matrix(Y_test, title_pred)

array([[553,  73],
       [239, 402]], dtype=int64)